In [4]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import normalize

In [16]:
def new_column_names(df_copy):
    column_list = ['label']
    row, column = 1,1;
    while row < 29:
        while column < 29:
            column_name = str(row) + 'x' + str(column)
            column_list.append(column_name)
            column+=1
        row+=1;
        if(column == 29):
            column = 1
            
    df_copy.columns = column_list
    return df_copy

def split_dataset_to_train_test_validation(df_copy, train_sze, test_sze, validation_sze):
    random.seed(0)
    target_column = ['label']
    predictors = list(set(list(df.columns))-set(target_column))
    X = df_copy[predictors] #Contains all columns
    Y = df_copy[['label']]
    X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=train_sze, random_state=50)
    X_test, X_validation, Y_test, Y_validation = train_test_split(X_temp, Y_temp, train_size=test_sze/(test_sze+validation_sze), random_state=50)
    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation

def normalizer(dataframe):
    result = dataframe.copy()/255
    return result

In [17]:
warnings.filterwarnings('ignore')
model = Sequential()
model.add(Dense(784, activation = 'sigmoid', name = 'Input_Layer'))
model.add(Dense(15, activation = 'sigmoid', name = 'Hidden_Layer_1'))
model.add(Dense(15, activation = 'sigmoid', name = 'Hidden_Layer_2'))
model.add(Dense(10))

In [18]:
df_train = pd.read_csv('MNIST_TRAIN.csv')
df_test = pd.read_csv('MNIST_TEST.csv')

In [19]:
print('Numbers of NaN values in training set: '); print(df_train.isnull().sum().sum()); 
print('Numbers of NaN values in test set: '); print(df_test.isnull().sum().sum());
print(df_train); print(df_test);

Numbers of NaN values in training set: 
0
Numbers of NaN values in test set: 
0
       label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0          5    0    0    0    0    0    0    0    0    0  ...      0      0   
1          0    0    0    0    0    0    0    0    0    0  ...      0      0   
2          4    0    0    0    0    0    0    0    0    0  ...      0      0   
3          1    0    0    0    0    0    0    0    0    0  ...      0      0   
4          9    0    0    0    0    0    0    0    0    0  ...      0      0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
59995      8    0    0    0    0    0    0    0    0    0  ...      0      0   
59996      3    0    0    0    0    0    0    0    0    0  ...      0      0   
59997      5    0    0    0    0    0    0    0    0    0  ...      0      0   
59998      6    0    0    0    0    0    0    0    0    0  ...      0      0   
59999      8    0    0    0    0    0   

In [23]:
test_column = ['label']
train_columns = list(set(list(df_train.columns))-set(test_column))

X_train = df_train[train_columns]
Y_train = df_train[test_column]

X_test = df_test[train_columns]
Y_test = df_test[test_column]

X_train = normalizer(X_train)
X_test = normalizer(X_test)

In [28]:
model.compile(loss = 'hinge')
model.fit(X_train, Y_train, epochs = 3)

Epoch 1/3
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0987
Epoch 2/3
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0987
Epoch 3/3
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0987


In [29]:
model.predict(X_test)

313/313 [==============================] - 1s 3ms/step


array([[7.9083347, 7.949467 , 7.9903893, ..., 7.9297385, 7.9249506,
        7.9555683],
       [3.6634128, 3.606351 , 3.4847872, ..., 3.6443071, 3.6721945,
        3.6461403],
       [4.5236425, 4.54261  , 4.49935  , ..., 4.526302 , 4.538836 ,
        4.5137043],
       ...,
       [7.538947 , 7.591529 , 7.610213 , ..., 7.558853 , 7.550881 ,
        7.5741987],
       [7.380696 , 7.4705586, 7.462289 , ..., 7.408673 , 7.3887224,
        7.425794 ],
       [6.294908 , 6.33504  , 6.300802 , ..., 6.2822485, 6.2540226,
        6.2847247]], dtype=float32)

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Dense)         (None, 784)               615440    
                                                                 
 Hidden_Layer_1 (Dense)      (None, 15)                11775     
                                                                 
 Hidden_Layer_2 (Dense)      (None, 15)                240       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
Total params: 627,615
Trainable params: 627,615
Non-trainable params: 0
_________________________________________________________________


In [49]:
from platform import python_version

In [50]:
python_version()

'3.10.5'

In [51]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      1.1.0
anyio                        3.6.1
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asttokens                    2.0.5
astunparse                   1.6.3
attrs                        21.4.0
Babel                        2.10.3
backcall                     0.2.0
beautifulsoup4               4.11.1
bleach                       5.0.0
cachetools                   5.2.0
certifi                      2022.6.15
cffi                         1.15.0
charset-normalizer           2.0.12
colorama                     0.4.5
cycler                       0.11.0
debugpy                      1.6.0
decorator                    5.1.1
defusedxml                   0.7.1
entrypoints                  0.4
executing                    0.8.3
fastjsonschema               2.15.3
flatbuffers                  1.12
fonttools                    4.33.3
gast                         0.4.0
g

In [52]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [53]:
tf.config.list_physical_devices('GPU')

[]